<a href="https://colab.research.google.com/github/nullpitch-dev/DS_L1_Notebooks/blob/master/DS_L1_EX_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

# loading data from a csv fle
url_purchase = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/ds_purchase_log.csv'
url_customer = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/ds_customer_mst.csv'
url_product = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/ds_product_mst.csv'

purchase = pd.read_csv(url_purchase)
customer = pd.read_csv(url_customer)
product = pd.read_csv(url_product)

In [0]:
customer = customer.sort_values(['cust_no', 'update_dt', 'age_gender'],
                                ascending=False)
base = purchase
# should be improved
base = base.assign(age_gender=base.apply( lambda x:
            customer[customer['cust_no'] == x['cust_no']]['age_gender'].iloc[0],
            axis=1))
base = pd.merge(base, product, how='left', on='item_no')

In [0]:
# [1]

base = base.assign(age=base.apply(lambda x: x['age_gender'].split()[0], axis=1))
base = base.assign(gender=base.apply(lambda x: x['age_gender'].split()[1], axis=1))

agg = base.groupby(['gender', 'item_ctg1']).agg({'item_amt': 'sum',
                                                 'cust_no': 'nunique'})
                                                 # counting unique cust_no only
agg = agg.assign(amtPerCust=agg['item_amt'] / agg['cust_no'])
agg = agg.sort_values(['gender', 'amtPerCust'], ascending=[False, False])

men = agg.loc['M'].iloc[1].name
women = agg.loc['F'].iloc[1].name

print(f'Answer [1] : Men = {men}, Women = {women}')

Answer [1] : Men = 의류, Women = 화장품


In [0]:
# [2]

import math

data1 = base.groupby('cust_no').agg({'item_amt': 'sum'})
data1 = data1.assign(rank=data1['item_amt'].rank(method='min', ascending=False))

cust_cnt = data1.shape[0]   # total number of customers

def highlow(x):
    if x['rank'] <= cust_cnt * 0.2:
        return 'High'
    elif x['rank'] >= cust_cnt * 0.8:
        return 'Low'
    else:
        return 'Middle'
        
# setting rank category (High or Low)
data1 = data1.assign(rank_c=data1.apply(lambda x: highlow(x), axis=1))

data1 = data1.groupby('rank_c').agg({'item_amt': 'sum'})
total = data1['item_amt'].sum()
high = data1.loc['High'].iloc[0]
low = data1.loc['Low'].iloc[0]

print(f'Answer [2] : High/Total = {math.floor(high / total * 100)}, ' + 
                   f'High/Low = {math.floor(high / low)}')

Answer [2] : High/Total = 49, High/Low = 9


In [0]:
# [3] using ols and anova_lm

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import MultiComparison
import math

data3 = base.groupby('cust_no').agg({'item_amt': 'sum',
                                     'age': lambda x: list(set(x))[0]})

def ageCat(x):
    if x['age'] == '30대':
        return 'G1'
    elif x['age'] == '40대':
        return 'G2'
    else:
        return 'G3'

data3 = data3.assign(age_cls=data3.apply(lambda x: ageCat(x), axis=1))

# performing ANOVA TEST
result = ols(formula='item_amt ~ C(age_cls)', data=data3).fit()
anova_table = anova_lm(result)
f_val = anova_table['F'][0]

# performing Tukey's Range Test
comparison = MultiComparison(data3['item_amt'], data3['age_cls'])
print(comparison.tukeyhsd())
# G2 > G1, G3 > G1, G2 > G3 so, G2 > G3 > G1
# (G2, G3) : True to reject, (G1, G3) False to reject
# conclusion : G2 > G3 = G1

print(f'\nAnswer [3] : F-value = {math.floor(f_val)}, ' +
      f"Tukey's Range Test = G2 > G3 = G1")

    Multiple Comparison of Means - Tukey HSD, FWER=0.05     
group1 group2  meandiff  p-adj    lower      upper    reject
------------------------------------------------------------
    G1     G2  8461.1735 0.001   4117.8207 12804.5262   True
    G1     G3   717.8831   0.9  -3454.2396  4890.0058  False
    G2     G3 -7743.2904 0.001 -12467.8902 -3018.6906   True
------------------------------------------------------------

Answer [3] : F-value = 11, Tukey's Range Test = G2 > G3 = G1


In [0]:
# [4]

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# finding customers who bought at least 2 sub categories
cust_list = base.groupby('cust_no').agg({'item_ctg2': 'nunique'})
cust_list = cust_list[cust_list['item_ctg2'] >= 2]
cust_list.reset_index(inplace=True)

# filtering purchase data within above customers
data2 = base[base['cust_no'].isin(cust_list['cust_no'])]

# transform customer - category - number of categories view
data2 = data2.groupby('cust_no').agg({'item_ctg2': lambda x: list(set(x))})
#data2 = data2.assign(cnt=data2.apply(lambda x: len(x['item_ctg2']), axis=1))

# item_ctg2 combinations in list
cat_list = list(data2['item_ctg2'])

# transform item_ctg2 combinations in case number X category view with True / False values
te = TransactionEncoder()
te_array = te.fit(cat_list).transform(cat_list)
te_df = pd.DataFrame(te_array, columns=te.columns_)

# find frequent items
freq_items = apriori(te_df, min_support=0.001, use_colnames=True)

# apply association rules
asso_rules = association_rules(freq_items, metric='confidence',
                               min_threshold=0.001)

# filter : antecedents is '농산물'
asso_rules = asso_rules.assign(check=asso_rules.apply(lambda x:
                                                      '농산물' in x['antecedents']
                                                      , axis=1))
asso_rules = asso_rules[asso_rules['check']]

# sort by lift in descending order
asso_rules.sort_values(by='lift', ascending=False)

best = list(asso_rules['consequents'].iloc[0])[0]

print(f'Answer [4] : Best category to show is {best}')


Answer [4] : Best category to show is 가공식품
